In [89]:
import xarray as xr
import arcpy
from arcpy.sa import *
import uuid

in_cube = r"C:\Users\Lewis\Desktop\test\cube.nc"

ds = xr.open_dataset(in_cube)
ds= ds.load()
ds.close()

ds =  (ds['nbart_nir'] - ds['nbart_red']) / (ds['nbart_nir'] + ds['nbart_red'])
ds = ds.to_dataset(name='veg_idx')

ds = xr.where(ds['veg_idx'] > 0.6, 1, 0)

ds = ds.sum('time')

df = ds.to_dataframe().to_records()

arcpy.env.addOutputsToMap = False

out_fc = 'temp'
srs = arcpy.SpatialReference(3577)
tmp_feat = arcpy.management.CreateFeatureclass(out_path='in_memory', 
                                               out_name=out_fc, 
                                               geometry_type='POINT', 
                                               spatial_reference=srs)

arcpy.AddField_management(in_table=tmp_feat, 
                          field_name='samples', 
                          field_type='LONG')

fields = ['samples', 'SHAPE@XY']
with arcpy.da.InsertCursor(tmp_feat, fields) as cursor:
    for point in df:
        row = [point[3], (point[1], point[0])]
        cursor.insertRow(row)
        

try:
    if arcpy.CheckExtension('Spatial') == 'Available':
        arcpy.CheckOutExtension('Spatial')
        
        arcpy.env.addOutputsToMap = True
        kdense = KernelDensity(in_features=tmp_feat, 
                               population_field='samples', 
                               search_radius=30,
                               area_unit_scale_factor='SQUARE_METERS')

        arcpy.CheckInExtension('Spatial')
        
    else:
        arcpy.AddError('Spatial Analyst extension could not be checked out.')
        raise #return

except:
    arcpy.AddError('Could not generate kernel density.')
    raise # return
    

<xarray.Dataset>
Dimensions:          (time: 308)
Coordinates:
    spatial_ref      int32 3577
  * time             (time) datetime64[ns] 2000-02-06T01:49:38 ... 2020-12-21...
Data variables: (12/16)
    veg_idx          (time) float32 0.3559 0.3384 0.2536 ... 0.2753 0.2663
    veg_clean        (time) float32 0.34 0.351 0.3586 ... 0.2886 0.2753 0.2663
    static_raw       (time) float32 0.0 0.0 0.0 0.0 0.0 ... -1.0 -1.0 -1.0 -1.0
    static_clean     (time) float32 0.0 0.0 0.0 0.0 0.0 ... -1.0 -1.0 -1.0 -1.0
    static_can_inc   (time) float32 nan nan nan nan nan ... nan nan nan nan nan
    static_can_dec   (time) float32 nan nan nan nan nan ... nan nan nan nan nan
    ...               ...
    dynamic_clean    (time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    dynamic_can_inc  (time) float32 nan nan nan nan nan ... nan nan nan nan nan
    dynamic_can_dec  (time) float32 nan nan nan nan nan ... nan nan nan nan nan
    dynamic_con_inc  (time) float32 nan nan nan nan nan ... nan nan nan nan nan
    dynamic_con_dec  (time) float32 nan nan nan nan nan ... nan nan nan nan nan
    dynamic_zones    (time) float32 nan nan nan nan nan ... nan nan nan nan nan

In [166]:
ds = ds.sum('time')

df = ds.to_dataframe().to_records()

arcpy.env.addOutputsToMap = False

out_fc = 'temp'
srs = arcpy.SpatialReference(3577)
tmp_feat = arcpy.management.CreateFeatureclass(out_path='in_memory', 
                                               out_name=out_fc, 
                                               geometry_type='POINT', 
                                               spatial_reference=srs)

arcpy.AddField_management(in_table=tmp_feat, 
                          field_name='samples', 
                          field_type='LONG')

fields = ['samples', 'SHAPE@XY']
with arcpy.da.InsertCursor(tmp_feat, fields) as cursor:
    for point in df:
        row = [point[3], (point[1], point[0])]
        cursor.insertRow(row)
        

try:
    if arcpy.CheckExtension('Spatial') == 'Available':
        arcpy.CheckOutExtension('Spatial')
        
        arcpy.env.addOutputsToMap = True
        kdense = KernelDensity(in_features=tmp_feat, 
                               population_field='samples', 
                               search_radius=30,
                               area_unit_scale_factor='SQUARE_METERS')

        arcpy.CheckInExtension('Spatial')
        
    else:
        arcpy.AddError('Spatial Analyst extension could not be checked out.')
        raise #return

except:
    arcpy.AddError('Could not generate kernel density.')
    raise # return
    